In [89]:
# !pip install dash

In [90]:
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [91]:
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv"
data = pd.read_csv(URL)

In [92]:
out = data
rec = data[data['Recession']==1]
yr = data[data['Year']==2006]

In [93]:
yearly_rec=rec.groupby('Year')['Automobile_Sales'].mean().reset_index()
figure=px.line(yearly_rec,x='Year',y='Automobile_Sales',title="Plot 1")
figure.show()        

In [94]:
nov_rec= rec.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
figure=px.bar(nov_rec,x='Vehicle_Type',y='Automobile_Sales',title='Plot 2')
figure.show()

In [95]:
exp_rec= rec.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
figure=px.pie(exp_rec,values='Advertising_Expenditure',names='Vehicle_Type',title="PLot 3")
figure.show()

In [96]:
eou_rec= rec.groupby(['unemployment_rate','Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
figure=px.bar(eou_rec,x='unemployment_rate',y='Automobile_Sales',color='Vehicle_Type',title='Plot 4')
figure.show()

In [97]:
yas= out.groupby('Year')['Automobile_Sales'].mean().reset_index()
figure=px.line(yas,x='Year',y='Automobile_Sales',title='Plot 1')
figure.show()

In [98]:
mon= yr.groupby('Month')['Automobile_Sales'].mean().reset_index()
figure=px.line(mon,x='Month',y='Automobile_Sales',title='Plot 2')
figure.show()

In [99]:
avr_vdata=yr.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
figure=px.bar(avr_vdata,x='Vehicle_Type',y='Automobile_Sales',title='Average Vehicles Sold by Vehicle Type in the year {}'.format(2006))
figure.show()        

In [100]:
expy= yr.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
figure=px.pie(expy,values='Advertising_Expenditure',names='Vehicle_Type',title="PLot 3")
figure.show()

In [101]:
app = dash.Dash(__name__)

In [102]:
year_list = [i for i in range(1980, 2024, 1)]
app.layout = html.Div(children=[html.H1('Automobile Sales Statistics Dashboard',style={'text-align': 'center', 'color': '#503D36', 'font-size': 24}),
             dcc.Dropdown(id='dropdown-statistics', 
                   options=[
                           {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                           {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
                           ],
                  placeholder='Select a report type',value='Select Statistics',
                  style={'text-align': 'center'}),
             dcc.Dropdown(id='select-year', 
                   options=[{'label': i, 'value': i} for i in year_list],
                  placeholder='Select Year',
                  style={'text-align': 'center'}),
             html.Div([html.Div(id='output-container', className='chart-grid', style={'display': 'flex','flex-direction':'column'})])],
                     style={'dispaly':'flex','flex-direction':'column','justify-content':'center','align-items':'center'})

In [103]:
@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    Input(component_id='dropdown-statistics',component_property='value'))
def update_input_container(stat):
    if stat =='Yearly Statistics': 
        return False
    else: 
        return True

In [104]:
@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='select-year', component_property='value'), Input(component_id='dropdown-statistics', component_property='value')])
def update_output_container(year, stat):
    df = data
    if stat == 'Recession Period Statistics':
        recession_data = df[df['Recession'] == 1]
        
        yearly_rec=recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(figure=px.line(yearly_rec,x='Year',y='Automobile_Sales',title="Automobile sales over Recession Period (year wise)"))
        
        nov_rec= recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2 = dcc.Graph(figure=px.bar(nov_rec,x='Vehicle_Type',y='Automobile_Sales',title='Average number of vehicles sold by vehicle type'))
        
        exp_rec= recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(figure=px.pie(exp_rec,values='Advertising_Expenditure',names='Vehicle_Type',title="Total expenditure share by vehicle type during recessions"))
        
        eou_rec= rec.groupby(['unemployment_rate','Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(figure=px.bar(eou_rec,x='unemployment_rate',y='Automobile_Sales',color='Vehicle_Type',title='Effect of unemployment rate on vehicle type and sales'))

        return [html.Div(className='chart-item', children=[html.Div(children=R_chart1),html.Div(children=R_chart2)],style={'display': 'flex'}),html.Div(className='chart-item', children=[html.Div(children=R_chart3),html.Div(children=R_chart4)],style={'display': 'flex'})]
    
    elif (year and stat =='Yearly Statistics') :
        yearly_data = data[data['Year'] == year]
        
        yas= data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(figure=px.line(yas,x='Year',y='Automobile_Sales',title='Yearly Automobile sales'))
        
        mon= yearly_data.groupby('Month')['Automobile_Sales'].mean().reset_index()
        Y_chart2 = dcc.Graph(figure=px.line(mon,x='Month',y='Automobile_Sales',title='Total Monthly Automobile sales in the year {}'.format(year)))
        
        avr_vdata=yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        Y_chart3 = dcc.Graph( figure=px.bar(avr_vdata,x='Vehicle_Type',y='Automobile_Sales',title='Average Vehicles Sold by Vehicle Type in the year {}'.format(year)))
        
        expy= yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(figure=px.pie(expy,values='Advertising_Expenditure',names='Vehicle_Type',title='Total Advertisement Expenditure for each vehicle in the year {}'.format(year)))
        
        return [html.Div(className='chart-item', children=[html.Div(children=Y_chart1),html.Div(children=Y_chart2)],style={'display': 'flex'}),html.Div(className='chart-item', children=[html.Div(children=Y_chart3),html.Div(children=Y_chart4)],style={'display': 'flex'})]           
                          
                          
    

In [105]:
if __name__ == '__main__':
    app.run_server()